In [1]:
from math import *;
import numpy as np;
from scipy import *;
import pandas as pd;
from keras.models import Sequential
import tensorflow as tf;

Using TensorFlow backend.


In [2]:
def initiateGrid():
    row_0=[0,0,0,0,0,9,0,6,7];
    row_1=[0,0,7,0,3,2,0,9,1];
    row_2=[6,0,2,7,1,0,0,0,0];
    row_3=[0,6,3,0,0,0,7,0,2];
    row_4=[7,0,0,0,0,0,0,0,8];
    row_5=[4,0,1,0,0,0,6,5,0];
    row_6=[0,0,0,0,2,1,8,0,5];
    row_7=[2,4,0,9,8,0,3,0,0];
    row_8=[1,7,0,5,0,0,0,0,0];
    return [row_0, row_1, row_2, row_3, row_4, row_5, row_6, row_7, row_8];

In [3]:
def splitGroup(grid, indices):
    values=[lis[:] for lis in indices];'copies by value'
    for i in range(len(indices)):
        for j in range(len(indices[i])):
            values[i][j]=grid[indices[i][j]];
    return values;

In [4]:
def findAllPeersIndices(rows_indices, columns_indices, boxes_indices):
    peers_indices=[];
    for x in range (81):
        row=int(x/9);
        column=x%9;
        box_index=int(column/3)+int(row/3)*3;
        
        boxes_peers_indices=boxes_indices[box_index];
        columns_peers_indices=columns_indices[column];
        rows_peers_indices=rows_indices[row];
        
        unique_peers_indices=list(set(boxes_peers_indices+columns_peers_indices+rows_peers_indices)-set([x]));
        peers_indices.append(unique_peers_indices);
        
    return peers_indices;

def findAllPeersValues(grid, peers_indices):
    peers_values=[];
    for i in range(81):
        peers=[];
        value=grid[i];
        for j in peers_indices[i]:
            peers.append(grid[j]);
        peers=list(set(peers));
        peers_values.append(peers);
    return peers_values

def findAllPeers(rows_indices, columns_indices, boxes_indices, rows_values, columns_values, boxes_values):
    peers_indices=[];
    peers_values=[];
    for x in range (81):
        row=int(x/9);
        column=x%9;
        if rows_values[row][column]!=0:
            peers_values.append([i for i in range(1,10)]);
            peers_indices.append([]);
            continue;
            
        box_index=int(column/3)+int(row/3)*3;
        boxes_peers_values=boxes_values[box_index];
        boxes_peers_indices=boxes_indices[box_index];
        columns_peers_values=columns_values[column];
        columns_peers_indices=columns_indices[column];
        rows_peers_values=rows_values[row];
        rows_peers_indices=rows_indices[row];
        
        unique_peers_values=list(set(boxes_peers_values+columns_peers_values+rows_peers_values));
        unique_peers_indices=list(set(boxes_peers_indices+columns_peers_indices+rows_peers_indices));
        peers_values.append(unique_peers_values);
        peers_indices.append(unique_peers_indices);
        
    return [peers_indices, peers_values];

In [5]:
def findCandidates(peers):
    all_values=[x for x in range(1,10)];
    return list(set(all_values)-set(peers));

In [6]:
def findAllCandidates(grid, peers_list):
    candidates_list=[];
    for i in range(len(grid)):
        if grid[i]!=0:
            candidates_list.append([]);
        else:
            peers=peers_list[i];
            candidates_list.append(findCandidates(peers));
    return candidates_list;

In [7]:
def checkNakedSingles(candidates_list):
    naked_singles_indices=[];
    naked_singles_values=[];
    for i in range(len(candidates_list)):
        if len(candidates_list[i])==1:
            naked_singles_indices.append(i);
            naked_singles_values.append(candidates_list[i][0]);
    return [naked_singles_indices, naked_singles_values]

In [8]:
def checkUniqueCandidatesPartial(indices_list, candidates_list):
    'this checks for a single row, column, or box'
    unique_candidates_indices=[];
    unique_candidates_values=[];
    counter=[[] for x in range(10)];
    for index in indices_list:
        candidates=candidates_list[index];
        for candidate in candidates:
            counter[candidate].append(index);
    for number in range(1,10):
        if len(counter[number])==1:
            unique_candidates_indices.append(counter[number][0]);
            unique_candidates_values.append(number);
    return [unique_candidates_indices, unique_candidates_values]

In [9]:
def checkUniqueCandidates(indices_list, candidates_list):
    unique_candidates_indices=[];
    unique_candidates_values=[];
    for indices in indices_list:
        [a, b]=checkUniqueCandidatesPartial(indices, candidates_list);
        unique_candidates_indices+=a;
        unique_candidates_values+=b;
    return [unique_candidates_indices, unique_candidates_values] 

In [10]:
def elementaryChecks(all_candidates, rows_indices, columns_indices, boxes_indices):
    indices=[];
    values=[];
    [checked_indices, checked_values]=checkNakedSingles(all_candidates);
    indices+=checked_indices;
    values+=checked_values;
    [checked_indices, checked_values]=checkUniqueCandidates(rows_indices, all_candidates);
    indices+=checked_indices;
    values+=checked_values;
    [checked_indices, checked_values]=checkUniqueCandidates(columns_indices, all_candidates);
    indices+=checked_indices;
    values+=checked_values;
    [checked_indices, checked_values]=checkUniqueCandidates(boxes_indices, all_candidates);
    indices+=checked_indices;
    values+=checked_values;
    return [indices, values]

In [11]:
def linearSolve(grid):
    while grid.count(0)>0:
        rows_values=splitGroup(grid, rows_indices);
        columns_values=splitGroup(grid, columns_indices);
        boxes_values=splitGroup(grid, boxes_indices);
        all_peers_values=findAllPeersValues(grid, all_peers_indices);
        all_candidates=findAllCandidates(grid, all_peers_values);
        [indices, values]=elementaryChecks(all_candidates, rows_indices, columns_indices, boxes_indices);
        if len(indices)==0:
            return grid;
        for i in range(len(indices)):
            grid[indices[i]]=values[i];
    return grid;

In [12]:
def checkContradiction(grid, all_peers_values, all_candidates):
    for i in range(81):
        value=grid[i];
        if value==0:
            if all_candidates[i]==[]:
                return 1;
        else:
            peers=all_peers_values[i];
            for peer in peers:
                if peer==value:
                    return 1;
    return 0;

def backPropagation(grid):
    rows_values=splitGroup(grid, rows_indices);
    columns_values=splitGroup(grid, columns_indices);
    boxes_values=splitGroup(grid, boxes_indices);
    all_peers_values=findAllPeersValues(grid, all_peers_indices);
    all_candidates=findAllCandidates(grid, all_peers_values);
    
    if grid.count(0)==0:
        return grid;
    if checkContradiction(grid, all_peers_values, all_candidates):
        return [];
    
    [indices, values]=elementaryChecks(all_candidates, rows_indices, columns_indices, boxes_indices);
    if len(indices)==0:
        len_list=[len(candidates) for candidates in all_candidates];
        len_list=np.array(len_list);
        len_list=len_list[len_list!=0];
        min_length=np.amin(len_list);
        min_index=np.where(len_list = min_length);
        for i in range(min_length):
            new_grid=grid[:];
            new_grid[min_index]=all_candidates[min_index][i];
            result=backPropagation(new_grid);
            if result!=[]:
                return result;
    else:
        for i in range(len(indices)):
            grid[indices[i]]=values[i];
        return backPropagation(grid);

In [13]:
data=pd.read_csv("sudoku.csv");
data=np.array(data);

In [14]:
'Initiation'
index=0;
rows_indices=[[j*9+i for i in range(9)] for j in range(9)];
columns_indices=[[j+i*9 for i in range(9)] for j in range(9)];
boxes_indices=[[i+j*9+i_offset*3+(j_offset*9)*3 for j in range(3) for i in range(3)]
               for j_offset in range(3) for i_offset in range(3)];

grid=[int(num) for num in data[index][0]];
sol_grid=[int(num) for num in data[index][1]];

columns_values=splitGroup(grid, columns_indices);
boxes_values=splitGroup(grid, boxes_indices);
all_peers_indices=findAllPeersIndices(rows_indices, columns_indices, boxes_indices);
all_peers_values=findAllPeersValues(grid, all_peers_indices);
all_candidates=findAllCandidates(grid, all_peers_values);

In [15]:
errors=0;
givens=[];
for index in range(1000):
    grid=[int(num) for num in data[index][0]];
    sol_grid=[int(num) for num in data[index][1]];
    givens.append(81-grid.count(0));

    columns_values=splitGroup(grid, columns_indices);
    boxes_values=splitGroup(grid, boxes_indices);
    all_peers_indices=findAllPeersIndices(rows_indices, columns_indices, boxes_indices);
    all_peers_values=findAllPeersValues(grid, all_peers_indices);
    all_candidates=findAllCandidates(grid, all_peers_values);
    
    ans_grid=linearSolve(grid);
    if ans_grid!=sol_grid:
        errors+=1;

In [16]:
def generatePuzzleRemoval(solvable_grid):
    filled_indices=[i for i in range(len(solvable_grid)) if solvable_grid[i]!=0];
    random_index=random.choice(filled_indices);
    new_grid=solvable_grid[:];
    new_grid[random_index]=0;
    
    rows_values=splitGroup(new_grid, rows_indices);
    columns_values=splitGroup(new_grid, columns_indices);
    boxes_values=splitGroup(new_grid, boxes_indices);
    all_peers_values=findAllPeersValues(new_grid, all_peers_indices);
    all_candidates=findAllCandidates(new_grid, all_peers_values);
    
    [indices, values]=elementaryChecks(all_candidates, rows_indices, columns_indices, boxes_indices);
    if len(indices)==0:
        return solvable_grid;
    else:
        return generatePuzzleRemoval(new_grid);

In [17]:
def hiddenPairElimination(group_indices, all_candidates):
    #first check if certain numbers occur in only two positions 
    #second check if a cell only contains two numbers, then note those two numbers down and check with remaining cells
    group_candidates=[all_candidates[i] for i in group_indices];
    pos_list=[[] for i in range(10)];
    
    for i in range(len(group_indices)):
        candidates=group_candidates[i];
        index=group_indices[i];
        for candidate in candidates:
            pos_list[candidate].append(index);
            
    
    pair_values=[i for i in range(1,10) if len(pos_list[i])==2];
    pair_indices=[pos_list[i] for i in range(1,10) if len(pos_list[i])==2];
    pair_values_dynamic=[];
    pair_indices_dynamic=[];
    
    for i in range(len(pair_values)):
        matched=0;
        value=pair_values[i];
        indices=pair_indices[i];
        for j in range(len(pair_values_dynamic)):
            if pair_indices_dynamic[j]==indices:
                keep=[value, pair_values_dynamic[j]];
                for index in indices:
                    all_candidates[index]=keep;
                del pair_indices_dynamic[j];
                del pair_values_dynamic[j];
                matched=1;
                break;
        if (matched==0):
            pair_values_dynamic.append(value);
            pair_indices_dynamic.append(indices);
    
            
    pair_values=[candidates for candidates in group_candidates if len(candidates)==2];
    pair_indices=[group_indices[i] for i in range(len(group_indices)) if len(group_candidates[i])==2];

In [18]:
def nakedDoubleElimination(group_indices, all_candidates):
    #first check if certain numbers occur in only two positions 
    #second check if a cell only contains two numbers, then note those two numbers down and check with remaining cells
    group_candidates=[all_candidates[i] for i in group_indices];
            
    pair_values=[candidates for candidates in group_candidates if len(candidates)==2];
    pair_indices=[group_indices[i] for i in range(len(group_indices)) if len(group_candidates[i])==2];
    pair_values_dynamic=[];
    pair_indices_dynamic=[];
    for i in range(len(pair_indices)):
        matched=0;
        values=pair_values[i];
        index=pair_indices[i];
        for j in range(len(pair_indices_dynamic)):
            if pair_values_dynamic[j]==values:
                keep_indices=[index, pair_indices_dynamic[j]];
                remove_indices=list(set(group_indices)-set(keep_indices));
                for index in remove_indices:
                    old_candidates=all_candidates[index][:];
                    all_candidates[index]=list(set(old_candidates)-set(values));
                del pair_indices_dynamic[j];
                del pair_values_dynamic[j];
                matched=1;
                break;
        if (matched==0):
            pair_values_dynamic.append(values);
            pair_indices_dynamic.append(index);

In [19]:
def pairElimination_Group(group_indices, all_candidates):
    hiddenPairElimination(group_indices, all_candidates);
    nakedDoubleElimination(group_indices, all_candidates);
    
def allPairElimination(rows_indices, columns_indices, boxes_indices, all_candidates):
    for i in range(len(rows_indices)):
        pairElimination_Group(rows_indices[i], all_candidates);
        pairElimination_Group(columns_indices[i], all_candidates);
        pairElimination_Group(boxes_indices[i], all_candidates);

In [20]:
def generatePuzzle(puzzle_grid, best_solution_grid):
    #print(count);
    rows_values=splitGroup(best_solution_grid, rows_indices);
    columns_values=splitGroup(best_solution_grid, columns_indices);
    boxes_values=splitGroup(best_solution_grid, boxes_indices);
    all_peers_values=findAllPeersValues(best_solution_grid, all_peers_indices);
    all_candidates=findAllCandidates(best_solution_grid, all_peers_values);
    if (checkContradiction(best_solution_grid, all_peers_values, all_candidates)):
        #contradiction
        return [[], []];
    if best_solution_grid.count(0)==0:
        #solved
        return [puzzle_grid, best_solution_grid];
    count_list=[[] for i in range(10)];
    for i in range(81):
        candidates=all_candidates[i];
        candidates_count=len(candidates);
        count_list[candidates_count].append(i);
    for i in range(9,0,-1):
        if count_list[i]!=[]:
            index_choice=random.choice(count_list[i]);
            break;
    new_puzzle_grid=puzzle_grid[:];
    new_best_solution_grid=best_solution_grid[:];
    candidate_choices=all_candidates[index_choice];
    end_puzzle_grid=[];
    while len(end_puzzle_grid)==0:
        if (len(candidate_choices))==0:
            return [[],[]];
        candidate_choice=random.choice(candidate_choices);
        candidate_choices.remove(candidate_choice);
        new_puzzle_grid[index_choice]=candidate_choice;
        new_best_solution_grid=best_solution_grid[:];
        new_best_solution_grid[index_choice]=candidate_choice;
        new_best_solution_grid=linearSolve(new_best_solution_grid);
        [end_puzzle_grid, end_solution_grid]=generatePuzzle(new_puzzle_grid, new_best_solution_grid);
    return [end_puzzle_grid, end_solution_grid];

In [21]:
def intermediateSolve(grid):
    while grid.count(0)>0:
        rows_values=splitGroup(grid, rows_indices);
        columns_values=splitGroup(grid, columns_indices);
        boxes_values=splitGroup(grid, boxes_indices);
        all_peers_values=findAllPeersValues(grid, all_peers_indices);
        all_candidates=findAllCandidates(grid, all_peers_values);
        allPairElimination(rows_indices, columns_indices, boxes_indices, all_candidates);
        [indices, values]=elementaryChecks(all_candidates, rows_indices, columns_indices, boxes_indices);
        if len(indices)==0:
            return grid;
        for i in range(len(indices)):
            grid[indices[i]]=values[i];
    return grid;

In [294]:
def generatePuzzleIntermediate(puzzle_grid, best_solution_grid):
    global branch_count;
    #print(count);
    rows_values=splitGroup(best_solution_grid, rows_indices);
    columns_values=splitGroup(best_solution_grid, columns_indices);
    boxes_values=splitGroup(best_solution_grid, boxes_indices);
    all_peers_values=findAllPeersValues(best_solution_grid, all_peers_indices);
    all_candidates=findAllCandidates(best_solution_grid, all_peers_values);
    if (checkContradiction(best_solution_grid, all_peers_values, all_candidates)):
        #contradiction
        return [[], []];
    if best_solution_grid.count(0)==0:
        #solved
        return [puzzle_grid, best_solution_grid];
    count_list=[[] for i in range(10)];
    for i in range(81):
        candidates=all_candidates[i];
        candidates_count=len(candidates);
        count_list[candidates_count].append(i);
    for i in range(9,0,-1):
        if count_list[i]!=[]:
            index_choice=random.choice(count_list[i]);
            break;
    new_puzzle_grid=puzzle_grid[:];
    new_best_solution_grid=best_solution_grid[:];
    candidate_choices=all_candidates[index_choice];
    end_puzzle_grid=[];
    while len(end_puzzle_grid)==0:
        branch_count+=1;
        if (len(candidate_choices))==0:
            return [[],[]];
        candidate_choice=random.choice(candidate_choices);
        candidate_choices.remove(candidate_choice);
        new_puzzle_grid[index_choice]=candidate_choice;
        new_best_solution_grid=best_solution_grid[:];
        new_best_solution_grid[index_choice]=candidate_choice;
        new_best_solution_grid=intermediateSolve(new_best_solution_grid);
        [end_puzzle_grid, end_solution_grid]=generatePuzzleIntermediate(new_puzzle_grid, new_best_solution_grid);
    return [end_puzzle_grid, end_solution_grid];

In [295]:
empty_grid=[0 for i in range(81)];
global branch_count;
branch_count=0;
[end_puzzle_grid, end_solution_grid]=generatePuzzleIntermediate(empty_grid, empty_grid);

In [296]:
print(branch_count)

27


In [297]:
print(81-end_puzzle_grid.count(0));
splitGroup(end_puzzle_grid, rows_indices)

24


[[7, 0, 1, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 6, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 7, 2, 3],
 [0, 0, 0, 5, 2, 0, 0, 0, 0],
 [6, 0, 4, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 3, 0, 0, 7, 0],
 [0, 1, 0, 0, 0, 0, 3, 0, 4],
 [0, 4, 8, 0, 0, 0, 9, 0, 0],
 [0, 0, 0, 0, 7, 9, 0, 0, 2]]

In [139]:
solution_grid_linear=linearSolve(end_puzzle_grid);
solution_grid=intermediateSolve(end_puzzle_grid);

In [140]:
solution_grid_linear==end_solution_grid

True

In [298]:
from PIL import Image
image_list=[];
for i in range(10):
    filename="Numbers/"+str(i)+".png";
    image=Image.open(filename);
    image_list.append(image);

In [299]:
def concatenateHorizontal(images):
    image_width=images[0].width;
    image_height=images[0].height;
    dst = Image.new('RGB', (image_width*len(images), image_height));
    for i in range(len(images)):
        dst.paste(images[i], (image_width*i, 0));
    return dst

def concatenateVertical(images):
    image_width=images[0].width;
    image_height=images[0].height;
    dst = Image.new('RGB', (image_width, image_height*len(images)));
    for i in range(len(images)):
        dst.paste(images[i], (0, image_height*i));
    return dst

In [300]:
rows_values=splitGroup(end_puzzle_grid, rows_indices);
#rows_values=[[0 for i in range(9)] for j in range(9)];
all_images=[];
for row in rows_values:
    row_image_list=[image_list[index] for index in row];
    all_images.append(concatenateHorizontal(row_image_list));
all_images.reverse;
output_image=concatenateVertical(all_images);

In [301]:
horizontal_line=Image.open("Numbers/HorizontalLine.png");
vertical_line=Image.open("Numbers/VerticalLine.png");

In [302]:
dimension=output_image.size[0]/3;
thickness=8;
horizontal_line=horizontal_line.resize((output_image.width, thickness));
vertical_line=vertical_line.resize((thickness, output_image.height));
output_image.paste(vertical_line, (int(dimension-thickness/2),0));
output_image.paste(vertical_line, (int(dimension*2-thickness/2),0));
output_image.paste(horizontal_line, (0,int(dimension-thickness/2)));
output_image.paste(horizontal_line, (0,int(dimension*2-thickness/2)));

In [304]:
output_image.save("Numbers/Puzzle1.png")